In [ ]:
import pandas as pd
import numpy as np
import matplotlib.ticker as ticker
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats


url = '/content/drive/MyDrive/Colab Notebooks/문제/excel/vgames2.csv'
df = pd.read_csv(url, index_col = 0)

# Year 정리
cond1 = (df['Year'] < 50)
cond2 = (df['Year'] >50) & (df['Year'] < 100)

df['Year'].loc[cond1] = df['Year'].loc[cond1] + 2000
df['Year'].loc[cond2] = df['Year'].loc[cond2] + 1900
df = df.loc[~(df['Year'] >= 2017)]
df['Year'].isna().sum() #na = 271

#sales 값 정리
#remove M, to_numeric
#remove K, to_numeric, *1/1000
def sales_to_numeric(col):
    col = col.replace('M','',regex = True
                      ).apply(pd.to_numeric,errors = 'ignore')
    condition_K = col.str.contains('K', na = False)
    contain_k = col.loc[condition_K]
    col.loc[condition_K] = contain_k.replace('K', '',regex = True
                                                                   ).apply(pd.to_numeric)*0.001
    
    return col

df['NA_Sales'] = sales_to_numeric(df['NA_Sales'])   
df['EU_Sales'] = sales_to_numeric(df['EU_Sales'])
df['JP_Sales'] = sales_to_numeric(df['JP_Sales'])
df['Other_Sales'] = sales_to_numeric(df['Other_Sales'])

df['Total_Sales'] = df['NA_Sales'] + df['EU_Sales'] + df['JP_Sales'] + df['Other_Sales']
df.iloc[:,5:] = df.iloc[:,5:].astype(float)

#Publischer df['Publisher'].loc[(df['Publisher'] == 'Unknown')]= 203
#Publischer na = 58
#replace na ,Unknown
df['Publisher'] = df['Publisher'].fillna('Unknown')


#중복 제거 (Madden NFL 13)
df.drop_duplicates(['Name', 'Platform','Year'], keep = 'first', inplace = True)
#Year 와 Genre의 na 값 drop
df.dropna(inplace = True)
df.reset_index(drop= True, inplace = True)

In [ ]:
np.corrcoef(df['Total_Sales'],df['JP_Sales'])
#>> 우선공략은 북미
# 왜 일본은 상관계수가 낮은가?
# 우선 시장이 상대적으로 작다.
# 그리고 선호장르영역이 다른 지역과 많이 다르다.


In [ ]:
df_Genre = df.pivot_table(values= 'Total_Sales', index = 'Genre',aggfunc= sum)
df_Genre['ratio'] = df_Genre['Total_Sales']/df_Genre['Total_Sales'].sum()
df_Genre

#1.지역에 따라서 선호하는 게임 장르가 다를까 (시각화, 카이스퀘어?)

In [ ]:
'''
def Genre_pie_chart(col_name):
    df_Genre = df.pivot_table(values= col_name, index = 'Genre',aggfunc= sum)
    df_Genre = df_Genre.sort_values(by = col_name)
    df_Genre_last = df_Genre.iloc[0:5]


    df_Genre.drop(df_Genre_last.index, inplace= True)
    df_Genre = df_Genre.T
    df_Genre['etc'] = df_Genre_last.sum()
    df_Genre = df_Genre.T

    G = df_Genre.plot.pie(y = col_name,
                          figsize = (8, 5.5),
                          autopct = "%.2f%%",
                          colors =sns.color_palette("hls", 8))
    plt.title(col_name)
    plt.legend(loc='center left', bbox_to_anchor=(1.2, 0.5))
    plt.show()
    return



Genre_pie_chart('NA_Sales')
Genre_pie_chart('EU_Sales')
Genre_pie_chart('JP_Sales')
Genre_pie_chart('Other_Sales')

#비율을 보여주려고 했는데 좋은 선택이 아니었다.
#우선 범주가 너무 많다 12개.
#그리고 범주별 색 설정이 복잡해진다. 범주가 많기 때문이다.
#또 범주별로 색을 지정해줘야 하는데 쉽게 가려다가 더 어렵게 꼬였다.
'''

In [ ]:
def Genre_top(col_name): #지역마다 인기장르 4개 고르기
    df_Genre = df.pivot_table(values= col_name, index = 'Genre',aggfunc= sum)
    df_Genre = df_Genre.sort_values(by = col_name, ascending= False)
    df_Genre_top = df_Genre.iloc[0:4]

    return df_Genre_top

top_NA = Genre_top('NA_Sales')
top_EU = Genre_top('EU_Sales')
top_JP = Genre_top('JP_Sales')
top_Other = Genre_top('Other_Sales')

#인기장르 시각화
top_Total = pd.concat([top_NA, top_EU, top_JP,top_Other], axis = 1)
top_Total = top_Total.reset_index(drop= False)
top_Total.columns = ['Genre','NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales']
tidy = top_Total.melt(id_vars= 'Genre'  ).dropna()
tidy.columns = ['Genre', 'location','Sales' ]
sns.barplot(data = tidy, x = 'location', y ='Sales', hue = 'Genre',)

In [ ]:
#장르 분포 chi square 검정 (NA EU Other)

df_Genre2 = df.pivot_table(values= ['NA_Sales', 'EU_Sales', 'Other_Sales'],
                          index = 'Genre',
                          aggfunc= sum)
stats.chi2_contingency(df_Genre2) # p = 0.023
#유의수준 0.05에서 분포는 서로 다르다고 보는 대안가설 채택

In [ ]:
#장르 분포 chi square 검정 (NA EU Other)
df_Genre3 = df.pivot_table(values= ['NA_Sales', 'EU_Sales'],
                          index = 'Genre',
                          aggfunc= sum)
stats.chi2_contingency(df_Genre3) # p = 0.031 다르다고 본다.

In [ ]:
카이스퀘어 굳이 써야할까? 
1. 일본은 충분히 다르다.
궁금한 것은 
>> NA EU OTHER >>다르다!
>>.NA EU>> 다르다!

In [ ]:
'''
g4 = sns.barplot(data=df, x = 'Genre', y = 'Other_Sales', estimator= sum)
g4.set_xticklabels(g4.get_xticklabels(), rotation=45);
'''
#비교해서 보기 불편하다.

#2.연도별 게임의 트렌드가 있을까

In [ ]:
#연도 뽑아내기
Genre_pivot = df.pivot_table(values= 'Total_Sales' ,index = 'Genre', columns = 'Year',aggfunc= sum).fillna(0)
Genre_pivot = Genre_pivot.iloc[:,20:] 

In [ ]:
#그래프 그리기 1
from scipy import interpolate
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
%matplotlib inline

def add_widths(x, y, width=0.1):
    """ Adds flat parts to widths """
    new_x = []
    new_y = []
    for i,j in zip(x,y):
        new_x += [i-width, i, i+width]
        new_y += [j, j, j]
    return new_x, new_y

def bumpsplot(dataframe, color_dict=defaultdict(lambda: "k"), 
                         linewidth_dict=defaultdict(lambda: 1),
                         labels=[]):
    r = dataframe.rank(method="first")
    r = (r - r.max() + r.max().max()).fillna(0) # Sets NAs to 0 in rank
    for i in r.index:
        x = np.arange(r.shape[1])
        y = r.loc[i].values
        color = color_dict[i]
        lw = linewidth_dict[i]
        x, y = add_widths(x, y, width=0.1)
        xs = np.linspace(0, x[-1], num=1024)
        plt.plot(xs, interpolate.PchipInterpolator(x, y)(xs), color=color, linewidth=lw, alpha=0.5)
        if i in labels:
            plt.text(x[0] - 0.1, y[0], s=i, horizontalalignment="right", verticalalignment="center", color=color)
            plt.text(x[-1] + 0.1, y[-1], s=i, horizontalalignment="left", verticalalignment="center", color=color)
    plt.xticks(np.arange(r.shape[1]), dataframe.columns)

In [ ]:
#그래프 그리기 2
Genre_colors = defaultdict(lambda: "grey")
lw = defaultdict(lambda: 1)

top_Genre = Genre_pivot.iloc[:, 0].dropna().sort_values().index
for i,c in enumerate(top_Genre):
    Genre_colors[c] = sns.color_palette("husl", n_colors=len(top_Genre))[i]
    lw[c] = 4

plt.figure(figsize=(24,12))
bumpsplot(Genre_pivot, color_dict=Genre_colors, linewidth_dict=lw, labels=top_Genre)
plt.gca().get_yaxis().set_visible(False)

In [ ]:
'''
#line은 너무 난잡하다
plt.figure(figsize = (30,20))
df1996 = df.loc[df['Year']>=1996]
sns.lineplot(data = df, x = 'Year', y = 'Total_Sales', hue = 'Genre', ci = None, estimator= sum)
'''

In [ ]:
'''
#장르가 너무 많다. 다른 방법..추천
g22 = sns.FacetGrid(df, row = 'Genre')
g22.map(sns.lineplot,'Year','Total_Sales', ci = None, estimator = sum)
'''

In [ ]:
'''
sns.lineplot(data = df, x = "Year", y = 'Total_Sales',ci = None)
#게임당 평균 판매량은 감소,횡보
'''

In [ ]:
# 전체시장이 감소중
fig2 = plt.figure(figsize= (10,7))
ax = sns.barplot(data = df, x = "Year", y = 'Total_Sales',ci = None, estimator= sum)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45);

plt.show()

In [ ]:
'''
sns.countplot(data = df, x = 'Year')
#게임 갯수는 시장상황에 맞게 늘어나고 줄어들음
'''

In [ ]:
'''
#정규성은 찾아보기 힘들다.
from scipy import stats

stats.normaltest(df['EU_Sales'])
'''

#3.출고량이 높은 게임에 대한 분석 및 시각화 프로세스

In [ ]:
#이름은 같지만 플랫폼이 다른 게임들의 total Sales 구하기
df_without_dupl = df.loc[~df.duplicated(['Name'], keep = False)].sort_values(by = 'Name')
df_dupl = df.loc[df.duplicated(['Name'], keep = False)].sort_values(by = 'Name')
df_dupl_grouped = df_dupl.groupby(df_dupl['Name']).sum().reset_index()
df_dupl_Sales = df_dupl_grouped[['Name', 'Total_Sales']]
df_without_dupl_Sales = df_without_dupl[['Name', 'Total_Sales']]

df_Sales = pd.concat([df_without_dupl_Sales, df_dupl_Sales])
df_Sales = df_Sales.sort_values(by = 'Total_Sales', ascending= False).reset_index(drop = True)


In [ ]:
#elbow method
'''
inertia_arr = []
k_range = range(2, 15)

for k in k_range:

    Kmeans2 = KMeans(n_clusters = k, random_state =10)
    Kmeans2.fit(Z)
    interia = Kmeans2.inertia_

    #print('k:', k,'inertia:',interia)

    inertia_arr.append(interia)

inertia_arr = np.array(inertia_arr)

plt.plot(k_range, inertia_arr)
#k = 4 또는 5
'''

In [ ]:
#우선 outlier제거
out= df_Sales.loc[df_Sales['Total_Sales']>= 40]
dfin = df_Sales.drop(out.index)
dfin.reset_index(inplace= True,drop= True)


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
#Kmeans k = 5
# Total Sales에 대하여 KMeans를 시행하여 상위 집단을 고른다.
# elbow method를 통해 4 또는 5정도가 적합하다
# Kmeans는 2d 이상만 받으므로 x=y에 놓여있다고 생각하기로 한다
#KMeans는 outlier에 민감하기 때문에, 미리 제거한다.(Wii Sports 등 3개)
scaler = StandardScaler()
df_tt = pd.concat([dfin['Total_Sales'], dfin['Total_Sales']], axis = 1)
Z = scaler.fit_transform(df_tt)
kmeans = KMeans(n_clusters= 5 ,random_state= 10)
K = kmeans.fit_transform(Z)
klabel = pd.DataFrame(kmeans.labels_)
klabel.columns = ['Label']

#k값이 여러차원으로 나오기 때문에 차원축소를 위해 PCA를 시행
pca = PCA(n_components= 2)
P = pca.fit_transform(K)

p_column = ['PC1','PC2','label']
df_p = pd.DataFrame(P)
df_p = pd.concat([df_p, klabel], axis =1)
df_p.columns = p_column
sns.scatterplot(data = df_p, x = 'PC1', y= 'PC2', hue = 'label')
# array([0.70508712, 0.26103036])# 96%의 분산을 설명한다.

In [ ]:
dfin_label = pd.concat([dfin, klabel], axis = 1)
df_top1 = dfin_label.loc[dfin_label['Label']== 3]
df_top1 = df_top1.append(out)#이상치 추가

df_top2 = dfin_label.loc[dfin_label['Label']== 1]
df_top3 = dfin_label.loc[dfin_label['Label']== 4]



In [ ]:
#상위권 게임 이름들
a = df_top1.sort_values(by = 'Name').iloc[:16].reset_index()
b=  df_top1.sort_values(by = 'Name').iloc[16:]
b= b.T
b['151'] = None
b = b.T.reset_index()


c = pd.concat([a['Name'],b['Name']], axis = 1)
c
#31개의 개임.
#높은 충성도
#시리즈 게임
# 같은 게임사들

In [ ]:
#이름으로 해당 데이터 가져옴
df_top11 = df.loc[df['Name'].isin(df_top1['Name'])]
df_top22 = df.loc[df['Name'].isin(df_top2['Name'])]
df_top33 = df.loc[df['Name'].isin(df_top3['Name'])]

In [ ]:
'''
g = sns.barplot(data = df_top11, x = 'Year', y = 'Total_Sales', estimator= sum, ci = None)
g.set_xticklabels(g.get_xticklabels(), rotation=45);
''

In [ ]:
#상위 게임들의 개발사 이름 찾기
df_pub_list3 = pd.concat([df_top11['Publisher'],
                         df_top22['Publisher'],
                         df_top33['Publisher']],
                        axis = 0).unique()
df_pub_list3 = np.delete(df_pub_list3,np.where(df_pub_list3 =='Unknown'))

df_pub_list2 = pd.concat([df_top11['Publisher'],
                         df_top22['Publisher']],
                        axis = 0).unique()

df_pub_list1 = df_top11['Publisher'].unique()

In [ ]:
#상위권 게임사들의 판매 점유율 구하기
def top_publisher_Sales_ratio(df,df_pub_list):
    
    Pub_Sales = df.loc[df['Publisher'].isin(df_pub_list)]['Total_Sales'].sum()
    Total_Sales = df['Total_Sales'].sum()
    ratio = Pub_Sales/Total_Sales

    return ratio
    
r_array = np.array([top_publisher_Sales_ratio(df, df_pub_list1),
                    top_publisher_Sales_ratio(df, df_pub_list2),
                    top_publisher_Sales_ratio(df, df_pub_list3)])

xt = np.array(['top6', 'top14', 'top44'])
colors = sns.color_palette("hls", 3)
gg = plt.bar(xt , r_array, color = colors)


#4. 지니계수와 시장의 크기를 고려한 장르 선정하기

In [ ]:
2012~2016지니계수 구하기
최근 5년간 해당 장르의 총 판매량/지니계수
상대적으로 평등하면서 시장의 크기도 고려한 지표


In [ ]:
'''
#total에 대한 상대도수 분석
df_total = df.sort_values(by ='Total_Sales')
df_total['Total_Sales']
cumratio = np.cumsum(df_total['Total_Sales'])/df_total['Total_Sales'].sum()

sns.lineplot(data = cumratio, x = np.arange(0,len(cumratio))/len(cumratio), y = cumratio)
(0.5 - (cumratio/len(cumratio)).sum())/0.5
'''

In [ ]:
from sklearn.preprocessing import MinMaxScaler
#시장크기 스케일링(음수값은 의미 없어보여서 min max로 설정, 0이 되는 값은 어차피 시장의 크기가 너무 작으므로 제외가능)


df_g = df.loc[df['Year']>= 2012]
df_g_Genre = df_g.pivot_table(values= 'Total_Sales', index = 'Genre',aggfunc= sum)

#구분구적으로 구함 엄밀히 따지면 조금 다른 지니계수.
def gini(genre_str):
    df_gg = df_g.loc[df_g['Genre'] == genre_str].sort_values(by ='Total_Sales')
    cumratio = np.cumsum(df_gg['Total_Sales'])/df_gg['Total_Sales'].sum()
    gini = (0.5 - (cumratio/len(cumratio)).sum())/0.5

    return gini

df_g_Genre.reset_index(inplace = True)
genre_list = df_g_Genre['Genre']

scaler_g = MinMaxScaler()
Z_g = scaler_g.fit_transform(df_g_Genre[['Total_Sales', 'Total_Sales']])

gini_by_size = Z_g.T[0]/np.array(genre_list.apply(gini))
df_g_Genre['gini'] = genre_list.apply(gini)
df_g_Genre['Sales_by_gini'] = gini_by_size

In [ ]:
#plot 장르별 지니계수와 총 판매량
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

matplotlib.rc_file_defaults()
ax1 = sns.set_style(style=None, rc=None )

fig, ax1 = plt.subplots(figsize=(12,6))
sns.barplot(data = df_g_Genre, x='Genre', y='Total_Sales')
ax2 = ax1.twinx()
sns.lineplot(data = df_g_Genre, x = 'Genre', y = 'gini')


In [ ]:
#두가지를 고려한 지표 플롯
gb =sns.barplot(data = df_g_Genre, x = 'Genre', y = 'Sales_by_gini')
gb.set_xticklabels(gb.get_xticklabels(), rotation=45);